In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import cohen_kappa_score
from statsmodels.stats.inter_rater import fleiss_kappa


In [14]:
data_deepseek_r1 = pd.read_parquet('data/Deepseek/DEEPSEEK_5_tables_final.parquet', engine='pyarrow')
data = pd.read_parquet('data/Mistral/data_mistral_complete.parquet', engine='pyarrow')
target_columns_names = [col for col in data if col.startswith('Mistral')]
data_for_plot = data[data['Translation_quality'] == 1.0].drop(['Translation_quality'], axis=1)[target_columns_names]

In [8]:
target_columns_names = [col for col in data_deepseek_r1 if col.startswith('Deep')]
data_deepseek_r1 = data_deepseek_r1[data_deepseek_r1['Translation_quality'] == 1.0].drop(['Translation_quality'], axis=1)[target_columns_names]

In [10]:
from itertools import combinations
from pprint import pprint
def compute_pairwise_cohen_kappas(df):
    kappas = {}
    for col1, col2 in combinations(df.columns, 2):
        valid_rows = df[[col1, col2]].dropna()
        kappa = cohen_kappa_score(valid_rows[col1], valid_rows[col2])
        kappas[(col1, col2)] = kappa
    return kappas

pairwise_kappas = compute_pairwise_cohen_kappas(data_deepseek_r1)
pprint(pairwise_kappas)


{('Deepseek_centrist_EN_answer', 'Deepseek_centrist_PL_answer'): np.float64(0.8152613627349885),
 ('Deepseek_centrist_EN_answer', 'Deepseek_centrist_RU_answer'): np.float64(0.778119237861094),
 ('Deepseek_centrist_PL_answer', 'Deepseek_centrist_RU_answer'): np.float64(0.8054733727810651),
 ('Deepseek_far_right_EN_answer', 'Deepseek_centrist_EN_answer'): np.float64(0.005275945852134756),
 ('Deepseek_far_right_EN_answer', 'Deepseek_centrist_PL_answer'): np.float64(0.02718401486988864),
 ('Deepseek_far_right_EN_answer', 'Deepseek_centrist_RU_answer'): np.float64(0.055118110236220486),
 ('Deepseek_far_right_EN_answer', 'Deepseek_far_right_PL_answer'): np.float64(0.8702659145850121),
 ('Deepseek_far_right_EN_answer', 'Deepseek_far_right_RU_answer'): np.float64(0.8679245283018868),
 ('Deepseek_far_right_EN_answer', 'Deepseek_mod_cons_EN_answer'): np.float64(0.027552674230145846),
 ('Deepseek_far_right_EN_answer', 'Deepseek_mod_cons_PL_answer'): np.float64(0.08272528698238601),
 ('Deepseek_fa

In [15]:
from itertools import combinations
from pprint import pprint
def compute_pairwise_cohen_kappas(df):
    kappas = {}
    for col1, col2 in combinations(df.columns, 2):
        valid_rows = df[[col1, col2]].dropna()
        kappa = cohen_kappa_score(valid_rows[col1], valid_rows[col2])
        kappas[(col1, col2)] = kappa
    return kappas

pairwise_kappas = compute_pairwise_cohen_kappas(data_for_plot)
pprint(pairwise_kappas)

{('Mistral_centrist_EN_answer', 'Mistral_centrist_PL_answer'): np.float64(0.009565945234963569),
 ('Mistral_centrist_EN_answer', 'Mistral_centrist_RU_answer'): np.float64(0.20882814250504145),
 ('Mistral_centrist_PL_answer', 'Mistral_centrist_RU_answer'): np.float64(0.05352181729040706),
 ('Mistral_far_right_EN_answer', 'Mistral_centrist_EN_answer'): np.float64(0.5397867104183757),
 ('Mistral_far_right_EN_answer', 'Mistral_centrist_PL_answer'): np.float64(0.014007760299205696),
 ('Mistral_far_right_EN_answer', 'Mistral_centrist_RU_answer'): np.float64(0.26219347221880607),
 ('Mistral_far_right_EN_answer', 'Mistral_far_right_PL_answer'): np.float64(0.3862368257904525),
 ('Mistral_far_right_EN_answer', 'Mistral_far_right_RU_answer'): np.float64(0.09304138782405313),
 ('Mistral_far_right_EN_answer', 'Mistral_mod_cons_EN_answer'): np.float64(0.5355504587155964),
 ('Mistral_far_right_EN_answer', 'Mistral_mod_cons_PL_answer'): np.float64(0.24413984015716383),
 ('Mistral_far_right_EN_answer',

In [13]:
def prepare_fleiss_matrix(df):
    fleiss_matrix = []
    for _, row in df.iterrows():
        valid = row.dropna()
        if len(valid) == df.shape[1]:  # Ensure full annotation
            counts = valid.value_counts().reindex([0, 1], fill_value=0)
            fleiss_matrix.append(counts.tolist())
    return np.array(fleiss_matrix)


fleiss_data = prepare_fleiss_matrix(data_deepseek_r1)
fleiss_kappa_value = fleiss_kappa(fleiss_data)
print("Fleiss' kappa (all annotators):", fleiss_kappa_value)


Fleiss' kappa (all annotators): 0.25546155243557506


In [16]:
def prepare_fleiss_matrix(df):
    fleiss_matrix = []
    for _, row in df.iterrows():
        valid = row.dropna()
        if len(valid) == df.shape[1]:  # Ensure full annotation
            counts = valid.value_counts().reindex([0, 1], fill_value=0)
            fleiss_matrix.append(counts.tolist())
    return np.array(fleiss_matrix)


fleiss_data = prepare_fleiss_matrix(data_for_plot)
fleiss_kappa_value = fleiss_kappa(fleiss_data)
print("Fleiss' kappa (all annotators):", fleiss_kappa_value)

Fleiss' kappa (all annotators): 0.157365653224452
